### Ejemplo del taxista
[Hoja de cálculo](https://docs.google.com/spreadsheets/d/1RHwkhmq2qKMV5LIjMt43g0VHawsT6M0-rjk0UhjqsC4/edit?usp=sharing)

Deseamos conocer la facturación que raliza un taxi en cuatro carreras en las que ha realizado estos kilometrajes: 10, 4, 25, 20.

Sabemos que:
1. Por bajada de bander se cobra 2,5 euros
2. Por kilómetro recorrido se cobra 1,3 euros

La ecuación para calcular el importe de una carrera de taxi es una línea recta:
$$y = 2,5 + 1,3 x$$

donde:
- $x$ son los kilómetros recorridos
- $y$ es el importe de la carrera, lo que se ha de pagar

Al compararlo con el producto matricial que se hace en un perceptrón al multiplicar la matriz de pesos $W$ por los valores de la variable independiente $X$, se añade un 1 correspondiente al **bias** o sesgo. Así la recta puede verse como:

$$y = 1 \cdot 2,5 + 1,3 \cdot x$$

Podemos resolverlo de forma matricial, para lo cual necesitamos añadir una columna de unos a la matriz $W$.

El 1 hace referencia a que en cada carrera únicamente se cobra una vez la bajada de bandera.

$$Y = W \cdot X$$

Con nuestros datos:

$$Y = \begin{bmatrix}
1 & 10 \\
1 & 4 \\
1 & 25 \\
1 & 20
\end{bmatrix}_{4 \times 2}
\begin{bmatrix}
2.5 \\
1.3
\end{bmatrix}_{2 \times 1}
=
\begin{bmatrix}
15.5 \\
7.7 \\
35 \\
28.5
\end{bmatrix}_{4 \times 1}$$

Para calcular la facturación total simplemente sumamos los importes de estas cuatro carreras.

Facturación Total = 15,50 + 7,70 + 35,00 + 28,50 = 86,70 €.

In [ ]:
import numpy as np

# Definir el array con los kilómetros recorridos por el taxi en 4 carreras
X = np.array([10, 4, 25, 20])

# Ampliar X añadiendo una fila de unos (para la bajada de bandera)
X_ampliado = np.vstack([np.ones(X.shape), X])

# Definir los pesos, bajada de bandera y precio por kilómetro recorrido
W = np.array([2.5, 1.3])

# Multiplicamos las dos matrices
# Calcular el vector: Importes facturados en cada carrera
importes = np.dot(W, X_ampliado)

print("X_ampliado:")
print(X_ampliado)
print("\nVector: Importes facturados en cada carrera")
print(importes)
print("\nTotal Facturado:")
print(np.sum(importes))

X_ampliado:
[[ 1.  1.  1.  1.]
 [10.  4. 25. 20.]]

Vector: Importes facturados en cada carrera
[15.5  7.7 35.  28.5]

Total Facturado:
86.7


#### Numpy es flexible al multiplicar
- Si multiplicamos usando la W Transpuesta no da error ya que Numpy es flexible
- Además el resultado que se obtiene es idéntico. Numpy se encarga de encajar las dimensiones

In [4]:
print(np.dot(W, X_ampliado))
print(np.dot(W.T, X_ampliado))  # usamos la transpuesta de W

[22.25  7.7  37.25 33.  ]
[22.25  7.7  37.25 33.  ]


## Añadimos una nueva variable
Consideremos que en la tarifa del taxi también hay que añadir 0,50 € por cada minuto de espera.

La nueva ecuación para calcular el importe de una carrera de taxi también es lineal, pero ahora es un plano en 3D.

$$y = 2,5 + 1,3 x_1 + 0,5 x_2$$

donde:
- $x_1$ son los kilómetros recorridos
- $x_2$ son los minutos de espera
- $y$ es el importe de la carrera

La ecuación matricial no varía, pero ahora las matrices son mayores.

$$Y = W \cdot X$$

$$Y = \begin{bmatrix}
1 & 10 & 15\\
1 & 4 & 0\\
1 & 25 & 5\\
1 & 20 & 10
\end{bmatrix}_{4 \times 3}
\begin{bmatrix}
2.5 \\
1.3 \\
0.5
\end{bmatrix}_{3 \times 1}
=
\begin{bmatrix}
23 \\
7.7 \\
37.5 \\
33.5
\end{bmatrix}_{4 \times 1}$$

Para calcular la facturación total simplemente sumamos los importes de estas cuatro carreras.

Facturación Total = 23,00 + 7,70 + 37,50+ 33,50 = 101,70 €.

In [5]:
import numpy as np

# Definir la matriz con los km. y minutos de las 4 carreras del taxi
X = np.array([[10, 4, 25, 20],  # kilómetros recorridos
              [15, 0, 5, 10]])  # minutos de espera

# Ampliar X añadiendo una fila de unos (para la bajada de bandera)
X_ampliado = np.vstack([np.ones(X.shape[1]), X])

# Pesos: bajada de bandera, precio por kilómetro recorrido y minuto de espera
W = np.array([2.5, 1.3, 0.5])

# Multiplicamos las dos matrices
# Calcular el vector: Importes facturados en cada carrera
importes = np.dot(W, X_ampliado)

print("X_ampliado:")
print(X_ampliado)

print("\nVector: Importes facturados en cada carrera")
print(importes)

print("\nTotal Facturado:")
print(np.sum(importes))

X_ampliado:
[[ 1.  1.  1.  1.]
 [10.  4. 25. 20.]
 [15.  0.  5. 10.]]

Vector: Importes facturados en cada carrera
[23.   7.7 37.5 33.5]

Total Facturado:
101.7
